In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

import pandas as pd

In [73]:
dataGen = pd.read_excel('data.xlsx', sheet_name = 'Gen')
dataLoad = pd.read_excel('data.xlsx', sheet_name = 'Load')
Ng = len(dataGen)
dataGen

,ID,PgLimit,PgCost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [75]:
model = pyo.ConcreteModel()
model.Pg = pyo.Var(range(Ng), bounds = (0,None))
Pg = model.Pg
Pg_sum = sum([Pg[id] for id in dataGen.ID])
model.Balance = pyo.Constraint(expr = Pg_sum == sum(dataLoad.Pd))
model.Condition = pyo.Constraint(expr = Pg[0] + Pg[3] >= dataLoad.Pd[0])
model.Limits = pyo.ConstraintList()
for id in dataGen.ID:
    model.Limits.add(expr = Pg[id] <= dataGen.PgLimit[id])
objective = sum([
                Pg[id] * dataGen.PgCost[id]
                for id in dataGen.ID
            ])
model.obj = pyo.Objective(expr = objective)
model.pprint()

1 Var Declarations
    Pg : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]

3 Constraint Declarations
    Balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    Condition : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  50.0 : Pg[0] + Pg[3]

In [76]:
opt = SolverFactory('glpk')
results = opt.solve(model)

In [78]:
model.pprint()

1 Var Declarations
    Pg : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  20.0 :  None : False : False :  Reals
          1 :     0 :  10.0 :  None : False : False :  Reals
          2 :     0 :  35.0 :  None : False : False :  Reals
          3 :     0 :  30.0 :  None : False : False :  Reals
          4 :     0 :   5.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]

3 Constraint Declarations
    Balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    Condition : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  50.0 : Pg[0] + Pg[3]

In [79]:
dataGen['Pg'] = [
                    pyo.value(Pg[id])
                    for id in dataGen.ID
                ]
dataGen

,ID,PgLimit,PgCost,Pg
0,0,20,0.10,20.0
1,1,10,0.05,10.0
2,2,40,0.30,35.0
3,3,50,0.40,30.0
4,4,5,0.01,5.0


In [80]:
model.Pg.pprint()

Pg : Size=5, Index={0, 1, 2, 3, 4}
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  20.0 :  None : False : False :  Reals
      1 :     0 :  10.0 :  None : False : False :  Reals
      2 :     0 :  35.0 :  None : False : False :  Reals
      3 :     0 :  30.0 :  None : False : False :  Reals
      4 :     0 :   5.0 :  None : False : False :  Reals


In [81]:
model.Condition.pprint()

Condition : Size=1, Index=None, Active=True
    Key  : Lower : Body          : Upper : Active
    None :  50.0 : Pg[0] + Pg[3] :  +Inf :   True


In [97]:
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]


In [82]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 25.05
  Upper bound: 25.05
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 5
  Number of nonzeros: 12
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.006585836410522461
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [99]:
pyo.value(Pg[0])

20.0

In [102]:
pyo.value(objective).item()

25.05